# VARIABLES

In [4]:
T = 12
Inventory_Carrying_Cost = 100 #for per ton 
Max_Over_Time = 20 #per month per worker maximum overtime
Workforce = 100 #bir shiftteki total kisi sayisi
Regular_Time_Salary = 10
Over_Time_Salary = 15
Production_Unit_Cost = 1000 #per ton
Initial_Inventory = 150 #ton
Constant_Inventory_Level = 100 #ton
hours_per_shift = 8
shifts_per_day = 2
working_days_per_t = 20
final_inventory = 150
Fixed_Regular_Working_Hours = hours_per_shift * shifts_per_day * working_days_per_t * Workforce
k = 100 #Manhours required to produce one ton of product
PHorizon = [1,2,3,4,5,6,7,8,9,10,11,12]
PHorizonAug = [2,3,4,5,6,7,8,9,10,11,12]
demand = {1:280,2:301,3:277,4:310,5:285,6:278,7:291,8:220,9:304,10:295,11:302,12:297}



## LP MODELLING

#### Decision Variables 

In [5]:
from pulp import *
AggPlan=LpProblem("Agg_Plan_Assignment1",LpMinimize)
X = LpVariable.dicts("Production",PHorizon,0)
I = LpVariable.dicts("Inventory",PHorizon,0)
O = LpVariable.dicts("Overtime_Working_Hours",PHorizon,0)

### Objective Function

In [6]:


Total_prod_cost = lpSum(Production_Unit_Cost * X[t] for t in PHorizon)
Total_inventory_cost  = lpSum(Inventory_Carrying_Cost * I[t] for t in PHorizon)
Total_labor_cost = lpSum((Regular_Time_Salary * Fixed_Regular_Working_Hours + Over_Time_Salary * O[t])  for t in PHorizon )
        
AggPlan += Total_prod_cost + Total_inventory_cost + Total_labor_cost

### Constraints

In [7]:


AggPlan += X[1]+Initial_Inventory - demand[1]-I[1] == 0 #"Balance"

for t in PHorizonAug:
    AggPlan += X[t]+I[t-1]-demand[t]-I[t] == 0  # Balance 
    
for t in PHorizon:
    AggPlan += (k * X[t]) <= Fixed_Regular_Working_Hours+O[t] #Capacity 
    
for t in PHorizon:
    AggPlan += O[t] <= Max_Over_Time * Workforce * shifts_per_day #Overtime

for t in PHorizon:
    AggPlan += I[t] >= Constant_Inventory_Level

AggPlan += I[12] == final_inventory

# Solution For Question 1 

In [8]:
AggPlan.solve()
# The status of the solution is printed to the screen
print("Status:", LpStatus[AggPlan.status])

# OUTPUT

# The optimised objective function value is printed to the console
print("Total Optimal Cost = ", pulp.value(AggPlan.objective))

# Each of the variables is printed with its resolved optimum value
for v in AggPlan.variables():
    print(v.name, "=", v.varValue)
#production = [v.varValue for v in gemstoneprob.variables()]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kaan/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/cd8456785e4d420998dc55231a74d9c8-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/cd8456785e4d420998dc55231a74d9c8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 54 COLUMNS
At line 175 RHS
At line 225 BOUNDS
At line 226 ENDATA
Problem MODEL has 49 rows, 36 columns and 84 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 22 (-27) rows, 33 (-3) columns and 54 (-30) elements
Perturbing problem by 0.001% of 1500 - largest nonzero change 0.0031285477 ( 0.0011986835%) - largest zero change 0
0  Obj 998701.07 Primal inf 2593.3 (11) Dual inf 899.99957 (1)
19  Obj 3568603.1
Optimal - objective value 3568600
After Postsolve, objec

# Question 2

In [9]:


Price = 2600 #per ton
# Objective Function
AggPlan_2=LpProblem("Agg_Plan_Assignment1",LpMaximize)
Total_prod_cost = lpSum(Production_Unit_Cost * X[t] for t in PHorizon)
Total_inventory_cost  = lpSum(Inventory_Carrying_Cost * I[t] for t in PHorizon)
Total_labor_cost = lpSum((Regular_Time_Salary * Fixed_Regular_Working_Hours + Over_Time_Salary * O[t])  for t in PHorizon )
Revenue = lpSum(Price * X[t] for t in PHorizon)        
AggPlan_2 += Revenue - ( Total_prod_cost + Total_inventory_cost + Total_labor_cost)



# Constraints

AggPlan_2 += X[1]+Initial_Inventory - demand[1]-I[1] == 0 #"Balance"

for t in PHorizonAug:
    AggPlan_2 += X[t]+I[t-1]-demand[t]-I[t] == 0  # Balance 
    
for t in PHorizon:
    AggPlan_2 += (k * X[t]) <= Fixed_Regular_Working_Hours+O[t] #Capacity 
    
for t in PHorizon:
    AggPlan_2 += O[t] <= Max_Over_Time * Workforce * shifts_per_day #Overtime

for t in PHorizon:
    AggPlan_2 += I[t] >= Constant_Inventory_Level

AggPlan_2 += I[12] == final_inventory

AggPlan_2.solve()
# The status of the solution is printed to the screen
print("Status:", LpStatus[AggPlan_2.status])
# OUTPUT

# The optimised objective function value is printed to the console
print("Total Profit = ", pulp.value(AggPlan_2.objective))

# Each of the variables is printed with its resolved optimum value
for t in PHorizon:
    print(f"Month {t}: Production = {X[t].varValue}, Overtime = {O[t].varValue}, Inventory = {I[t].varValue} ")   

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kaan/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/43386240ad8b45c6aafa81fe88f31e42-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/43386240ad8b45c6aafa81fe88f31e42-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 54 COLUMNS
At line 175 RHS
At line 225 BOUNDS
At line 226 ENDATA
Problem MODEL has 49 rows, 36 columns and 84 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 22 (-27) rows, 33 (-3) columns and 54 (-30) elements
Perturbing problem by 0.001% of 1700 - largest nonzero change 0.0060929727 ( 0.0012908899%) - largest zero change 0
0  Obj 1272917.7 Primal inf 2593.3 (11) Dual inf 17499.983 (11)
21  Obj 5375393.9
Optimal - objective value 5375400
After Postsolve, 

# Question 3 and 4
 

In [10]:
demand_list = [280, 301, 277, 310, 285, 278, 291, 220, 304, 295, 302, 297]
def optimize_production_and_calculate_profit(t, new_demand): # We define a funstion to calculate total profit
    Price = 2600
    promotion_month = t -1
    # Define the problem
    AggPlan_3 = LpProblem("Maximize_Profit", LpMaximize)



    # Decision Variables
    X = LpVariable.dicts("Production", PHorizon, 0)
    I = LpVariable.dicts("Inventory", PHorizon, 0)
    O = LpVariable.dicts("Overtime_Working_Hours", PHorizon, 0)

    # Objective Function
    AggPlan_3 += lpSum([(Price - (260 if t == promotion_month else 0)) * X[t] -
                        (Production_Unit_Cost * X[t] +
                         Regular_Time_Salary * hours_per_shift * shifts_per_day * working_days_per_t * Workforce +
                         Over_Time_Salary * O[t] +
                         Inventory_Carrying_Cost * I[t]) for t in PHorizon])
    
    # Constraints
    AggPlan_3 += X[1] + Initial_Inventory - new_demand[0] - I[1] == 0  # Balance for the first month

    for t in PHorizonAug:
        AggPlan_3 += X[t] + I[t-1] - new_demand[t-1] - I[t] == 0  # Balance for subsequent months
    
    for t in PHorizon:
        AggPlan_3 += (k * X[t]) <= Fixed_Regular_Working_Hours + O[t]  # Capacity 
    
    for t in PHorizon:
        AggPlan_3 += O[t] <= Max_Over_Time * Workforce * shifts_per_day  # Overtime

    for t in PHorizon:
        AggPlan_3 += I[t] >= Constant_Inventory_Level  # Inventory level constraint

    AggPlan_3 += I[12] == final_inventory  # End inventory constraint

    # Solve the model
    AggPlan_3.solve()
    # The status of the solution is printed to the screen
    print("Status:", LpStatus[AggPlan_3.status])
    # Print the results
    print(f"Optimal production plan and profit for promotion in month {promotion_month + 1}:")
    for t in PHorizon:
        print(f"Month {t}: Production = {X[t].varValue}, Overtime = {O[t].varValue}, Inventory = {I[t].varValue} ")
    print(f"Total Profit: ${value(AggPlan_3.objective)}")

# Adjusted demands for promotions in April and August
new_demand_april = [280, 301, 277, 577.6, 228.0, 222.4] + demand_list[6:]       #we changed demand list manually just for this question
new_demand_august = demand_list[:7] + [449.8, 243.2, 236] + demand_list[10:]





# Optimize production and calculate profit for promotions in April and August
print("####################### SOLUTION Q4 #######################")
optimize_production_and_calculate_profit(4, new_demand_april)  
print("####################### SOLUTION Q5 #######################")
optimize_production_and_calculate_profit(8, new_demand_august)  


####################### SOLUTION Q4 #######################
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kaan/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/488999ed5ebe4d49b441a431a09ddb92-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/488999ed5ebe4d49b441a431a09ddb92-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 54 COLUMNS
At line 175 RHS
At line 225 BOUNDS
At line 226 ENDATA
Problem MODEL has 49 rows, 36 columns and 84 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 22 (-27) rows, 33 (-3) columns and 54 (-30) elements
Perturbing problem by 0.001% of 1700 - largest nonzero change 0.0048007025 ( 0.0012908899%) - largest zero change 0
0  Obj 2598257.6 Primal inf 1843.6001 (11) Dual inf 17239.988 (11)
22  Ob

# Question 5

We have 4 condition. 

Condition 1 : If Q&H promotes, Unilock doesn't

Condition 2 : If Unilioc promotes, Q&H doesn't

Condition 3 : If both firms promote

Condition 4 : If both firms don't promote

In [11]:
######### We have 4 cases and these cases affect firms' demand list 


############## Condition 1
def condition_1(t,liste):
    liste[t-1] = (liste[t-1] * 1.5 ) + (0.2 * liste[t]) + (liste[t + 1] * 0.2)  # promotion conditions 
    liste[t] = liste[t] * 0.8
    liste[t+1] = liste[t+1] * 0.8
    return liste


############## Condition 2
def condition_2(t,liste):
    liste[t-1] =  liste[t-1] * 0.5  
    
    return liste


############## Condition 3
def condition_3(t, demand_list):
    liste = demand_list.copy()  
    liste[t-1] =  demand_list[t-1] + (0.25 * demand_list[t]) + (demand_list[t + 1] * 0.25) 
    liste[t] = demand_list[t] * 0.75
    liste[t+1] = demand_list[t+1] * 0.75
    return liste

############## Condition 4
# nothing change  in the firms' inventory and production plan because of this condition

In [12]:
def promotion(t, new_demand,firm,event): # We define a promotion function (like profit function) to calculate profit with promotions
    Price = 2600
    months = ["January","February","March", "April", "May", "June","July", "August" ,"September","October","November","December"]

    month = t-1 

    if firm == "Q_H" :     #firms' selling price depends on promotion condition 
        if event == 1 or event == 3:
            promotion_month = t -1
        else:
            promotion_month = 0
    if firm == "Unilock" :
        if event == 2 or event == 3:
            promotion_month = t -1
        else:
            promotion_month = 0

    # Define the problem
    AggPlan_3 = LpProblem("Maximize_Profit", LpMaximize)

    condition = {1:"If Q&H promotes, Unilock doesn't",
                 2:"If Unilock promotes, Q&H doesn't",
                 3:"If both firms promote"}

    # Adjusted demand
    demand_list = new_demand

    # Decision Variables
    X = LpVariable.dicts("Production", PHorizon, 0)
    I = LpVariable.dicts("Inventory", PHorizon, 0)
    O = LpVariable.dicts("Overtime_Working_Hours", PHorizon, 0)

    # Objective Function
    AggPlan_3 += lpSum([(Price - (260 if t == promotion_month else 0)) * X[t] -
                        (Production_Unit_Cost * X[t] +
                         Regular_Time_Salary * hours_per_shift * shifts_per_day * working_days_per_t * Workforce +
                         Over_Time_Salary * O[t] +
                         Inventory_Carrying_Cost * I[t]) for t in PHorizon])
    
    # Constraints
    AggPlan_3 += X[1] + Initial_Inventory - demand_list[0] - I[1] == 0  # Balance for the first month

    for t in PHorizonAug:
        AggPlan_3 += X[t] + I[t-1] - demand_list[t-1] - I[t] == 0  # Balance for subsequent months
    
    for t in PHorizon:
        AggPlan_3 += (k * X[t]) <= Fixed_Regular_Working_Hours + O[t]  # Capacity 
    
    for t in PHorizon:
        AggPlan_3 += O[t] <= Max_Over_Time * Workforce * shifts_per_day  # Overtime

    for t in PHorizon:
        AggPlan_3 += I[t] >= Constant_Inventory_Level  # Inventory level constraint

    AggPlan_3 += I[12] == final_inventory  # End inventory constraint

    # Solve the model
    AggPlan_3.solve()
    # The status of the solution is printed to the screen
    print("Status:", LpStatus[AggPlan_3.status])
    # Print the results
    print(f"############################### Production And Inventory Plan for {firm} ###############################")
    print(f"Optimal production plan and profit for promotion in month {promotion_month + 1}:")
    for t in PHorizon:
        print(f"Month {t}: Production = {X[t].varValue}, Overtime = {O[t].varValue}, Inventory = {I[t].varValue} ")   

    print("############################### Profit ###############################")
    print(f"Total Profit: ${value(AggPlan_3.objective)} for {firm} {condition[event]} in {months[month]} ")
    return value(AggPlan_3.objective)  # we need to return total profit for comparing other probability 

### Solution for Question 5

In [13]:

demand_list = [280, 301, 277, 310, 285, 278, 291, 220, 304, 295, 302, 297]

# firm parameters 
Q = "Q_H"
U = "Unilock" 


# Adjusted demands for promotions in April
new_demand_april_5 = condition_2(4,demand_list)# April Demand  decreased  50%


# Optimize production and calculate profit for promotions in April when Q&H don't promotes, Unilock promotes
promotion(4, new_demand_april_5,Q,2) 

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kaan/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/d171682efeb4422aa04044a1540a5c5f-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/d171682efeb4422aa04044a1540a5c5f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 54 COLUMNS
At line 175 RHS
At line 225 BOUNDS
At line 226 ENDATA
Problem MODEL has 49 rows, 36 columns and 84 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 22 (-27) rows, 33 (-3) columns and 54 (-30) elements
Perturbing problem by 0.001% of 1700 - largest nonzero change 0.0060929727 ( 0.0012908899%) - largest zero change 0
0  Obj 1212277.8 Primal inf 2476.2 (11) Dual inf 17499.983 (11)
19  Obj 5127394
Optimal - objective value 5127400
After Postsolve, ob

1287400.0

# Question 6

In [14]:

demand_list = [280, 301, 277, 310, 285, 278, 291, 220, 304, 295, 302, 297]
new_demand_april_Q_H = condition_1(4,demand_list)
promotion(4, new_demand_april_Q_H,Q,1) 

print("##############################################################")

new_demand_april_Uni = condition_2(4, demand_list)

promotion(4,new_demand_april_Uni,U,1)


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kaan/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/5955d2f5006b4abaab6b5c604008a67b-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/5955d2f5006b4abaab6b5c604008a67b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 54 COLUMNS
At line 175 RHS
At line 225 BOUNDS
At line 226 ENDATA
Problem MODEL has 49 rows, 36 columns and 84 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 22 (-27) rows, 33 (-3) columns and 54 (-30) elements
Perturbing problem by 0.001% of 1700 - largest nonzero change 0.0048007025 ( 0.0012908899%) - largest zero change 0
0  Obj 2598257.6 Primal inf 1843.6001 (11) Dual inf 17239.988 (11)
22  Obj 5331015.1
Optimal - objective value 5331020
After Postsolv

1321320.0

# Question 7

In [15]:

demand_list = [280, 301, 277, 310, 285, 278, 291, 220, 304, 295, 302, 297]
new_demand_april_Q7 = condition_3(4,demand_list)

promotion(4, new_demand_april_Q7,Q,3) 

print("##############################################################")


promotion(4,new_demand_april_Q7,U,3)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kaan/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/b38faea01c6d4001bb007554040b1ad9-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/b38faea01c6d4001bb007554040b1ad9-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 54 COLUMNS
At line 175 RHS
At line 225 BOUNDS
At line 226 ENDATA
Problem MODEL has 49 rows, 36 columns and 84 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 22 (-27) rows, 33 (-3) columns and 54 (-30) elements
Perturbing problem by 0.001% of 1700 - largest nonzero change 0.0048007025 ( 0.0012908899%) - largest zero change 0
0  Obj 1651279.3 Primal inf 2321.6501 (11) Dual inf 17239.988 (11)
21  Obj 5264745.4
Optimal - objective value 5264750
After Postsolv

1424750.0

# Question 8

In [16]:

demand_list = [280, 301, 277, 310, 285, 278, 291, 220, 304, 295, 302, 297]
new_demand_august_Q8 = condition_3(8,demand_list)

promotion(8, new_demand_august_Q8,Q,3) 

print("##############################################################")


promotion(8,new_demand_august_Q8,U,3)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kaan/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/cf523701424e49f994ef9c928357c52e-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/cf523701424e49f994ef9c928357c52e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 54 COLUMNS
At line 175 RHS
At line 225 BOUNDS
At line 226 ENDATA
Problem MODEL has 49 rows, 36 columns and 84 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 22 (-27) rows, 33 (-3) columns and 54 (-30) elements
Perturbing problem by 0.001% of 1700 - largest nonzero change 0.0053532753 ( 0.0012908899%) - largest zero change 0
0  Obj 1271953.6 Primal inf 2593.3 (11) Dual inf 17239.983 (11)
21  Obj 5290644.4
Optimal - objective value 5290650
After Postsolve, 

1450650.0

# Question 9

In [17]:

demand_list = [280, 301, 277, 310, 285, 278, 291, 220, 304, 295, 302, 297]
new_demand_Q9_Q_H = condition_1(4,demand_list)
new_demand_Q9_Q_H_yearly = condition_2(8,new_demand_Q9_Q_H)
print(f"Uptaded demand list for Q&H :{new_demand_Q9_Q_H_yearly} when Q&H promtes in april; Unilock promotes in August")
promotion(4, new_demand_Q9_Q_H_yearly,Q,1)  # we can use same function with parameter "4" because we have a uptaded demand list and price is just changed in April


Uptaded demand list for Q&H :[280, 301, 277, 577.6, 228.0, 222.4, 291, 110.0, 304, 295, 302, 297] when Q&H promtes in april; Unilock promotes in August
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kaan/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/dc5a269818d14030a26d01464959241f-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/dc5a269818d14030a26d01464959241f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 54 COLUMNS
At line 175 RHS
At line 225 BOUNDS
At line 226 ENDATA
Problem MODEL has 49 rows, 36 columns and 84 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 22 (-27) rows, 33 (-3) columns and 54 (-30) elements
Perturbing problem by 0.001% of 1700 - largest nonzero change 0.0048007025 ( 0.0012908899%) - lar

1315020.0

# Question 10

In [18]:

demand_list = [280, 301, 277, 310, 285, 278, 291, 220, 304, 295, 302, 297]
new_demand_august_Q10_Q_H = condition_1(8,demand_list)
new_demand_august_Q10_Q_H_yearly = condition_2(4,new_demand_august_Q10_Q_H)
print(f"Uptaded demand list for Q&H :{new_demand_august_Q10_Q_H_yearly} when Q&H promtes in august ; Unilock promotes in april")
promotion(8, new_demand_august_Q10_Q_H_yearly,Q,1)  # we can use same function with parameter "8" because we have a uptaded demand list and price is just changed in August


Uptaded demand list for Q&H :[280, 301, 277, 155.0, 285, 278, 291, 449.8, 243.20000000000002, 236.0, 302, 297] when Q&H promtes in august ; Unilock promotes in april
Status:Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kaan/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/2a90d4f15fda447c90c4eb9f92ceb25f-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/2a90d4f15fda447c90c4eb9f92ceb25f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 54 COLUMNS
At line 175 RHS
At line 225 BOUNDS
At line 226 ENDATA
Problem MODEL has 49 rows, 36 columns and 84 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 22 (-27) rows, 33 (-3) columns and 54 (-30) elements
Perturbing problem by 0.001% of 1700 - largest nonzero change 0.0053532753 (

1348880.0

jective 5188880, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 5188880 - 22 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



# For questions 11 and 12, we only need a function that will output total profit.

In [19]:

def total_profit_calculation(t, new_demand,firm,event):
    Price = 2600
    if firm == "Q_H" :     #firms' selling price depends on pormotion condition 
        if event == 1 or event == 3:
            promotion_month = t -1
        else:
            promotion_month = 0
    if firm == "Unilock" :
        if event == 2 or event == 3:
            promotion_month = t -1
        else:
            promotion_month = 0

    # Define the problem
    AggPlan_3 = LpProblem("Maximize_Profit", LpMaximize)

    condition = {1:"If Q&H promotes, Unilock doesn't",
                 2:"If Unilock promotes, Q&H doesn't",
                 3:"If both firms promote"}

    # Adjusted demand
    demand_list = new_demand

    # Decision Variables
    X = LpVariable.dicts("Production", PHorizon, 0)
    I = LpVariable.dicts("Inventory", PHorizon, 0)
    O = LpVariable.dicts("Overtime_Working_Hours", PHorizon, 0)

    # Objective Function
    AggPlan_3 += lpSum([(Price - (260 if t == promotion_month else 0)) * X[t] -
                        (Production_Unit_Cost * X[t] +
                         Regular_Time_Salary * hours_per_shift * shifts_per_day * working_days_per_t * Workforce +
                         Over_Time_Salary * O[t] +
                         Inventory_Carrying_Cost * I[t]) for t in PHorizon])
    
    # Constraints
    AggPlan_3 += X[1] + Initial_Inventory - demand_list[0] - I[1] == 0  # Balance for the first month

    for t in PHorizonAug:
        AggPlan_3 += X[t] + I[t-1] - demand_list[t-1] - I[t] == 0  # Balance for subsequent months
    
    for t in PHorizon:
        AggPlan_3 += (k * X[t]) <= Fixed_Regular_Working_Hours + O[t]  # Capacity 
    
    for t in PHorizon:
        AggPlan_3 += O[t] <= Max_Over_Time * Workforce * shifts_per_day  # Overtime

    for t in PHorizon:
        AggPlan_3 += I[t] >= Constant_Inventory_Level  # Inventory level constraint

    AggPlan_3 += I[12] == final_inventory  # End inventory constraint

    # Solve the model
    AggPlan_3.solve()
     # The status of the solution is printed to the screen
    print("Status:", LpStatus[AggPlan_3.status])

    return value(AggPlan_3.objective)

# Question 11


For this question, we thought that the two companies would coordinate to make promotions in one of August or April.


We calculated these probabilities in Question7, Question8, Question 9 and Question 10. 


So for this question we will compare these probabilities for Q&H company and see which is the optimal month selection.



In [20]:


demand_list = [280, 301, 277, 310, 285, 278, 291, 220, 304, 295, 302, 297]
# calculate total profit for Q&H if Q&H promotes in april and Unilock promotes in august.
# look at the scenarios in which Q&H chooses April and August respectively.


#  Q&H => April ; Unilock => August

total_profit_Q_H_april_Q11 = total_profit_calculation(4, new_demand_Q9_Q_H_yearly ,Q,1)  # we can use Q9 demand list 


#  Q&H => August ; Unilock => April

total_profit_Q_H_august_Q11 = total_profit_calculation(8, new_demand_august_Q10_Q_H_yearly,Q,1) # we can use Q10 demand list 

# Both firms => April 

total_profit_Q_H_april_both_Q11 = total_profit_calculation(4,new_demand_april_Q7,Q,3) # we can use Q7 demand list 

# Both firms => August 

total_profit_Q_H_august_both_Q11 = total_profit_calculation(8,new_demand_august_Q8,Q,3) # we can use Q8 demand list 

# we create list of all options
total_profits = [total_profit_Q_H_april_Q11, total_profit_Q_H_august_Q11, total_profit_Q_H_april_both_Q11, total_profit_Q_H_august_both_Q11] 

max_profit = max(total_profits)



print("####################### SOLUTION #######################")


if max_profit == total_profit_Q_H_april_Q11:
    print(f"Best option for Q&H: Q&H => April , Unilock => August and optimal profit = {total_profit_Q_H_april_Q11} ")
elif max_profit == total_profit_Q_H_august_Q11:
    print(f"Best option for Q&H: Q&H => August , Unilock => April  and optimal profit = {total_profit_Q_H_august_Q11}  ")
elif max_profit == total_profit_Q_H_april_both_Q11:
    print(f"Best option for Q&H: Both firms  => April  and optimal profit = {total_profit_Q_H_april_both_Q11} ")
else:
    print(f"Best option for Q&H: Both firms  => August  and optimal profit = {total_profit_Q_H_august_both_Q11} ")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kaan/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/96a3c0372f7848fc87c1bed1387f678b-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/96a3c0372f7848fc87c1bed1387f678b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 54 COLUMNS
At line 175 RHS
At line 225 BOUNDS
At line 226 ENDATA
Problem MODEL has 49 rows, 36 columns and 84 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 22 (-27) rows, 33 (-3) columns and 54 (-30) elements
Perturbing problem by 0.001% of 1700 - largest nonzero change 0.0048007025 ( 0.0012908899%) - largest zero change 0
0  Obj 2598257.6 Primal inf 1733.6001 (11) Dual inf 17239.988 (11)
22  Obj 5155015.2
Optimal - objective value 5155020
After Postsolv

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kaan/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/a8d17f34717b462bb8086d96c54788c7-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/a8d17f34717b462bb8086d96c54788c7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 54 COLUMNS
At line 175 RHS
At line 225 BOUNDS
At line 226 ENDATA
Problem MODEL has 49 rows, 36 columns and 84 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 22 (-27) rows, 33 (-3) columns and 54 (-30) elements
Perturbing problem by 0.001% of 1700 - largest nonzero change 0.0053532753 ( 0.0012908899%) - largest zero change 0
0  Obj 1201238.6 Primal inf 2586.2 (11) Dual inf 17239.983 (11)
22  Obj 5188874
Optimal - objective value 5188880
After Postsolve, ob

# Question 12


For this question, we thought that the two companies wouldn't coordinate to make promotions in one of August or April.


We calculated these probabilities in Question 3 and Question 4. 


So for this question we will compare these probabilities for Q&H company and see which is the optimal month selection without being affected by unilock company.



In [21]:


total_profit_Q_H_april_Q12 = total_profit_calculation(4,new_demand_april,Q,1) # We can use "new_demand_april" from Question 3
total_profit_Q_H_august_Q12 = total_profit_calculation(8,new_demand_august,Q,1) # We can use "new_demand_august" from Question 4

total_profits_Q12 = [total_profit_Q_H_august_Q12,total_profit_Q_H_april_Q12]

max_profit_Q12 = max(total_profits_Q12)



print("####################### SOLUTION #######################")
if max_profit_Q12 == total_profit_Q_H_april_Q12:
    print(f"Best option for Q&H: Q&H => April and optimal profit = {total_profit_Q_H_april_Q12} ")
elif max_profit_Q12 == total_profit_Q_H_august_Q12:
    print(f"Best option for Q&H: Q&H for Question12 => August and optimal profit = {total_profit_Q_H_august_Q12}" )


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kaan/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/b10625c899ba4df594a1d4165db4cc69-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/b10625c899ba4df594a1d4165db4cc69-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 54 COLUMNS
At line 175 RHS
At line 225 BOUNDS
At line 226 ENDATA
Problem MODEL has 49 rows, 36 columns and 84 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 22 (-27) rows, 33 (-3) columns and 54 (-30) elements
Perturbing problem by 0.001% of 1700 - largest nonzero change 0.0048007025 ( 0.0012908899%) - largest zero change 0
0  Obj 2598257.6 Primal inf 1843.6001 (11) Dual inf 17239.988 (11)
22  Obj 5331015.1
Optimal - objective value 5331020
After Postsolv

# Question 13

In this question, Q&H company decides to purchase detergent from a third-party external company instead of producing it.


This will cause us to make changes to the objective functions and constraints.


Instead of the production cost, we need to use the 3rd party company's sales price of $2,300.


We also need to remove labor-related capacities and variables from our LP model.

In [22]:


new_production_cost = 2300

def promotion_new(t, new_demand,firm,event):
    Price = 2600
    months = ["January","February","March", "April", "May", "June","July", "August" ,"September","October","November","December"]

    month = t-1 

    if firm == "Q_H" :     #firms' selling price depends on pormotion condition 
        if event == 1 or event == 3:
            promotion_month = t -1
        else:
            promotion_month = 0
    if firm == "Unilock" :
        if event == 2 or event == 3:
            promotion_month = t -1
        else:
            promotion_month = 0

    # Define the problem
    AggPlan_3 = LpProblem("Maximize_Profit", LpMaximize)

    condition = {1:"If Q&H promotes, Unilock doesn't",
                 2:"If Unilock promotes, Q&H doesn't",
                 3:"If both firms promote"}

    # Adjusted demand
    demand_list = new_demand

    # Decision Variables
       # We have just 2 decision variable because of Q&H  decides to purchase from a 3rd party company Instead of producing.
    X = LpVariable.dicts("Production", PHorizon, 0)
    I = LpVariable.dicts("Inventory", PHorizon, 0)


    # Objective Function
            # We don't have labor Cost
    
    AggPlan_3 += lpSum([(Price - (260 if t == promotion_month else 0)) * X[t] -
                        (new_production_cost * X[t] +
                         Inventory_Carrying_Cost * I[t]) for t in PHorizon])
    
    # Constraints
         # We don't have labor Capacity

    AggPlan_3 += X[1] + Initial_Inventory - demand_list[0] - I[1] == 0  # Balance for the first month

    for t in PHorizonAug:
        AggPlan_3 += X[t] + I[t-1] - demand_list[t-1] - I[t] == 0  # Balance for subsequent months
    for t in PHorizon:
        AggPlan_3 += I[t] >= Constant_Inventory_Level  # Inventory level constraint

    AggPlan_3 += I[12] == final_inventory  # End inventory constraint

    # Solve the model
    AggPlan_3.solve()

    # Print the results
    print(f"############################### Production And Inventory Plan for {firm} ###############################")
    print(f"Optimal production plan and profit for promotion in month {promotion_month + 1}:")
    for t in PHorizon:
        print(f"Month {t}: Production = {X[t].varValue}, Inventory = {I[t].varValue} ")   

    print("############################### Profit ###############################")
    print(f"Total Profit: ${value(AggPlan_3.objective)} for {firm} {condition[event]} in {months[month]} ")
    return value(AggPlan_3.objective)

## Question 13- (5-10)


We can use same demand list which was created before Question13 

In [23]:


print("####################### SOLUTION Q13-5 #######################")
promotion_new(4, new_demand_april_5,Q,2) 


print("####################### SOLUTION Q13-6 #######################")
promotion_new(4, new_demand_april_Q_H,Q,1) 



print("####################### SOLUTION Q13-7 #######################")

promotion_new(4, new_demand_april_Q7,Q,3) 



print("####################### SOLUTION Q13-8 #######################")
promotion_new(8, new_demand_august_Q8,Q,3) 


print("####################### SOLUTION Q13-9 #######################")
promotion_new(4, new_demand_Q9_Q_H_yearly,Q,1)


print("####################### SOLUTION Q13-10 #######################")

promotion_new(8, new_demand_august_Q10_Q_H_yearly,Q,1)



####################### SOLUTION Q13-5 #######################
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kaan/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/c184a84dd4bc49e69ea6ab44aab939a2-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/c184a84dd4bc49e69ea6ab44aab939a2-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 103 RHS
At line 129 BOUNDS
At line 130 ENDATA
Problem MODEL has 25 rows, 24 columns and 48 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-25) rows, 0 (-24) columns and 0 (-48) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 860500
After Postsolve, objective 860500, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 860500 - 0

864400.0

## Question 13- (11-12)


We can use same demand list which was created before Question13 

In [24]:

total_profit_Q_H_april_Q13 = promotion_new(4, new_demand_Q9_Q_H_yearly ,Q,1)  # we can use Q9 demand list 


#  Q&H => August ; Unilock => April:

total_profit_Q_H_august_Q13 = promotion_new(8, new_demand_august_Q10_Q_H_yearly,Q,1) # we can use Q10 demand list 

# Both firms => April :

total_profit_Q_H_april_both_Q13 = promotion_new(4,new_demand_april_Q7,Q,3) # we can use Q7 demand list 

# Both firms => August :

total_profit_Q_H_august_both_Q13 = promotion_new(8,new_demand_august_Q8,Q,3) # we can use Q8 demand list 

# we create list of all options:

total_profits_q13 = [total_profit_Q_H_april_Q13, total_profit_Q_H_august_Q13, total_profit_Q_H_april_both_Q13, total_profit_Q_H_august_both_Q13] 

max_profit = max(total_profits_q13)



print("####################### SOLUTION Q13-11 #######################")


if max_profit == total_profit_Q_H_april_Q13:
    print(f"Best option for Q&H: Q&H => April , Unilock => August and optimal profit = {total_profit_Q_H_april_Q13} ")
elif max_profit == total_profit_Q_H_august_Q13:
    print(f"Best option for Q&H: Q&H => August , Unilock => April  and optimal profit = {total_profit_Q_H_august_Q13}  ")
elif max_profit == total_profit_Q_H_april_both_Q13:
    print(f"Best option for Q&H: Both firms  => April  and optimal profit = {total_profit_Q_H_april_both_Q13} ")
else:
    print(f"Best option for Q&H: Both firms  => August  and optimal profit = {total_profit_Q_H_august_both_Q13} ")


####################### Question Q13-12 #######################


total_profit_Q_H_april_Q13_12 = total_profit_calculation(4,new_demand_april,Q,1) # We can use "new_demand_april" from Question 3
total_profit_Q_H_august_Q13_12 = total_profit_calculation(8,new_demand_august,Q,1) # We can use "new_demand_august" from Question 4

total_profits_Q13_12 = [total_profit_Q_H_august_Q13_12,total_profit_Q_H_april_Q13_12]

max_profit_Q13_12 = max(total_profits_Q13_12)



print("####################### SOLUTION Q13-12 #######################")

if max_profit_Q13_12 == total_profit_Q_H_april_Q13_12:
    print(f"Best option for Q&H: Q&H => April and optimal profit = {total_profit_Q_H_april_Q13_12} ")
elif max_profit_Q13_12 == total_profit_Q_H_august_Q13_12:
    print(f"Best option for Q&H: Q&H for Question12 => August and optimal profit = {total_profit_Q_H_august_Q13_12}" )
    

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kaan/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/7537929bf56842f9a6cbba94ced796fb-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/fw/gt8mwzcx2vqcdqxdq3gnmqrc0000gn/T/7537929bf56842f9a6cbba94ced796fb-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 30 COLUMNS
At line 103 RHS
At line 129 BOUNDS
At line 130 ENDATA
Problem MODEL has 25 rows, 24 columns and 48 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-25) rows, 0 (-24) columns and 0 (-48) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 892800
After Postsolve, objective 892800, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 892800 - 0 iterations time 0.002, Presolve 0.00
Option for printingOption